In [ ]:
#src = gdal.Open('/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/oh/oh.tif')
src = gdal.Open('/media/philipp/5e9929a4-cb93-48b8-8648-6c83a93e83ed/tmp/raster_1_temp.tif')
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
lrx = ulx + (src.RasterXSize * xres)
lry = uly + (src.RasterYSize * yres)

print('upper left corner: ', ulx, uly)
print('lower right corner: ', lrx, lry)

In [ ]:
from osgeo import ogr

# Create ring
ring = ogr.Geometry(ogr.wkbLinearRing)
ring.AddPoint(ulx, uly)
ring.AddPoint(ulx, lry)
ring.AddPoint(lrx, lry)
ring.AddPoint(lrx, uly)

# Create polygon
poly = ogr.Geometry(ogr.wkbPolygon)
poly.AddGeometry(ring)

print(poly.ExportToWkt())

In [ ]:
import osgeo.ogr as ogr
import osgeo.osr as osr

# set up the shapefile driver
driver = ogr.GetDriverByName("ESRI Shapefile")

# create the data source
data_source = driver.CreateDataSource("test.shp")

# create the spatial reference, WGS84
srs = osr.SpatialReference()
srs.ImportFromEPSG(31287)

# create the layer
layer = data_source.CreateLayer("poly", srs, ogr.wkbPolygon)

#field_region = ogr.FieldDefn("Region", ogr.OFTString)
#field_region.SetWidth(24)
#layer.CreateField(ogr.FieldDefn("Latitude", ogr.OFTReal))
#layer.CreateField(ogr.FieldDefn("Longitude", ogr.OFTReal))
layer.CreateField(ogr.FieldDefn('id', ogr.OFTInteger))
defn = layer.GetLayerDefn()

# Create a new feature (attribute and geometry)
feature = ogr.Feature(defn)
feature.SetField('id', 123)

# Make a geometry, from Shapely object
geom = ogr.CreateGeometryFromWkb(poly.ExportToWkt())
feature.SetGeometry(geom)

# Create the feature in the layer (shapefile)
layer.CreateFeature(feature)
# Dereference the feature
feature = None

# Save and close the data source
data_source = None